<a href="https://colab.research.google.com/github/priiiya3/Credicxo/blob/master/Credicxo_Assignment_Priya_Singh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import time
from json.decoder import JSONDecodeError
from urllib.error import HTTPError

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [10]:
# set the header and proxy
header = ({'User-Agent':'Mozilla/5.0 (X11; Linux x86_64)AppleWebKit/537.36 (KHTML, like Gecko)Chrome/44.0.2403.157 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})

sess = requests.Session()
sess.proxies = {"http": "http://65.243.12.170:92"}


In [ ]:
# Reading URLs from the given cvs file
prod_url = []
prod_file = pd.read_csv('https://docs.google.com/spreadsheets/d/' + 
                   '1BZSPhk1LDrx8ytywMHWVpCqbm8URTxTJrIRkD7PnGTM' +
                   '/export?gid=0&format=csv', index_col=0)

# loop to iterate through all the countries and asin.
for i, field in prod_file.iterrows():
	country = field['country']
	asin = field['Asin']
	prod_url.append('https://www.amazon.{}/dp/{}'.format(country, asin))


program_starts = time.time()
# this list stores all products
products = []

for i in prod_url:
    src = sess.get(i, headers=header)

    #log check to find 404 urls
    # print("i",i)
    # print("src", src)
    soup = BeautifulSoup(src.content, 'lxml')

    # if extracted source product has no content available then skip
    if not src.content:
        continue
    
    # to handle the url not found errors
    error_element = soup.find('a', href="/ref=cs_404_logo/")
    if error_element:
        print(f"{i} is Not Found")
        continue

    # this dictionary contains the information of a single product
    prod_dict = {}

    # To fetch Product Title
    prod_title = soup.find("span", attrs={"id": 'productTitle'})
    if prod_title is not None:
        prod_dict['Title'] = prod_title.text.strip()

    # To fetch Product Image URL
    prod_image = soup.find("img", attrs={"id": 'landingImage'})
    if prod_image is not None:
        prod_dict['Image'] = prod_image.attrs["src"]
    
    # To fetch Price of the Product
    prod_price = soup.find("span", attrs={"class": 'a-offscreen'})
    if prod_price is not None:
        prod_dict['Price'] = prod_price.text.strip()

    # To fetch Product Details
    prod_details = ''
    for info in soup.findAll("span", attrs={"class": 'a-list-item'}):
        if info.string is not None:
            details_value = info.text.strip()
            if len(details_value) != 1:
                prod_details += details_value + ' '
    if prod_details != '':
        prod_dict['Details'] = prod_details
    
    # add product details to products list
    if prod_dict != {}:
        products.append(prod_dict)


In [7]:
# To store the output data
with open('result.json', 'r+') as fp:
    try:
        file_data = json.load(fp)
        file_data += products
        fp.seek(0)
        json.dump(file_data, fp, indent=4)
    except JSONDecodeError:
        json.dump(products, fp, indent=4)

# Logging the execution time
now = time.time()
print("Total Time Elasped: {}".format(now - program_starts))


Total Time Elasped: 151.26480793952942
